## 1. Imports


In [12]:
import os
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns;

In [13]:
data_path = '../data/results/test_notbalanced_and_balanced/'
#folders= ['dt','gnb','lgr','gbt']
folders= ['dt']

## 2. CSVs

In [14]:
def impact_csvs(data_path= 'data/results/',b_or_w = 'Black', folders= ['dt','gnb','lgr','gbt']):

    col_names_eg = []
    col_names_gs = []

    for i,f in enumerate(folders):
        if b_or_w == 'Black':
            path = f'{data_path}{f}/{f}_black_results.csv'
        else:
            path = f'{data_path}{f}/{f}_white_results.csv'

        df = pd.read_csv(path,index_col=0)
        df = df.reset_index()

        col_names_eg.append(f'EG+{f.upper()}')
        col_names_gs.append(f'GS+{f.upper()}')

        if i == 0:
            joined_df = df.iloc[:,-1]
        else:
            joined_df = pd.concat([joined_df, df.iloc[:,-1]], axis=1)

    joined_df.set_axis(folders, axis=1)


    # split dataframe after the two reduction algorithms
    df_eg = joined_df.iloc[:6,:]
    df_gs = pd.concat([joined_df.iloc[0:1,:],joined_df.iloc[6:,:]])

    # set new index
    df_eg['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
    df_eg.set_index('Constraint',inplace=True)
    df_gs['Constraint'] = ['Unmitigated', 'DP', 'EO', 'EOO','FPER','ERP']
    df_gs.set_index('Constraint',inplace=True)
    df_eg.columns = col_names_eg
    df_gs.columns = col_names_gs

    df_final = pd.concat([df_eg, df_gs], axis=1)
    print('Group: ',b_or_w,'\n DataFrame: \n',df_final)
    print('A')
    df_final.to_csv(f'{data_path}/{b_or_w}_DI.csv')
    print('B')


In [15]:
impact_csvs(data_path,'Black', folders= ['dt','gnb','lgr','gbt'])
impact_csvs(data_path,'White', folders= ['dt','gnb','lgr','gbt'])

FileNotFoundError: [Errno 2] No such file or directory: '../data/results/test_notbalanced_and_balanced/gnb/gnb_black_results.csv'

## 3. FP/TP/TN/FN Ratios

In [16]:
# Ratios
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts(normalize=True)
    df = df.reset_index()
    df = df.rename(columns= {0:'Ratio'})
    
    df = df.pivot(index='Category', columns='ID')['Ratio']
    df = df.fillna(0)
    df = df.round(decimals = 3)
    print('Classifier: ',f,'\n DataFrame: \n',df)
    df.to_csv(f'{data_path}{f}/{f}_type_ratios.csv')

Classifier:  dt 
 DataFrame: 
 ID        egdpB  egdpW  egeoB  egeoW  egerpB  egerpW  egfprpB  egfprpW  \
Category                                                                 
FN        0.012  0.035  0.058  0.051   0.143   0.044    0.062    0.035   
FP        0.331  0.129  0.131  0.122   0.045   0.153    0.121    0.129   
TN        0.169  0.371  0.369  0.378   0.455   0.347    0.379    0.371   
TP        0.488  0.465  0.442  0.449   0.357   0.456    0.438    0.465   

ID        egtprpB  egtprpW  gsdpB  gsdpW  gseoB  gseoW  gserpB  gserpW  \
Category                                                                 
FN          0.045    0.035    0.0  0.051  0.143  0.029   0.143   0.029   
FP          0.189    0.128    0.5  0.098  0.045  0.145   0.045   0.145   
TN          0.311    0.372    0.0  0.402  0.455  0.355   0.455   0.355   
TP          0.455    0.465    0.5  0.449  0.357  0.471   0.357   0.471   

ID        gsfprpB  gsfprpW  gstprpB  gstprpW  testB  testW  unmitB  unmitW  
Ca

In [17]:
# Absolute numbers
dfs = {} # list for pandas dfs
pd.set_option('display.max_columns', None)
for i,f in enumerate(folders):
    path = f'{data_path}{f}/{f}_all_types.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    df = df.melt(var_name="ID",value_name="Category")
    df = df.groupby('ID').value_counts()
    df = df.reset_index()
    df = df.rename(columns= {0:'Number'})
    
    df = df.pivot(index='Category', columns='ID')['Number']
    df = df.fillna(0)
    print('Classifier: ',f,'\n DataFrame: \n',df)
    df.to_csv(f'{data_path}{f}/{f}_type_absolute.csv')

Classifier:  dt 
 DataFrame: 
 ID         egdpB   egdpW   egeoB   egeoW  egerpB  egerpW  egfprpB  egfprpW  \
Category                                                                     
FN          28.0   439.0   140.0   636.0   344.0   553.0    150.0    439.0   
FP         799.0  1608.0   315.0  1522.0   109.0  1907.0    293.0   1608.0   
TN         407.0  4622.0   891.0  4708.0  1097.0  4323.0    913.0   4622.0   
TP        1178.0  5791.0  1066.0  5594.0   862.0  5677.0   1056.0   5791.0   

ID        egtprpB  egtprpW   gsdpB   gsdpW   gseoB   gseoW  gserpB  gserpW  \
Category                                                                     
FN          109.0    442.0     0.0   637.0   344.0   364.0   344.0   364.0   
FP          456.0   1595.0  1206.0  1218.0   109.0  1804.0   109.0  1804.0   
TN          750.0   4635.0     0.0  5012.0  1097.0  4426.0  1097.0  4426.0   
TP         1097.0   5788.0  1206.0  5593.0   862.0  5866.0   862.0  5866.0   

ID        gsfprpB  gsfprpW  gst

# Analyzing Scores

#### Extractig Scores from csv into dataframes

In [18]:
# Scores Data Frames
classifier_dfs = {}
dfs_b = {}
dfs_w = {}
dfs_eg = {}
dfs_gs = {}
for f in folders:
    path = f'{data_path}{f}/{f}_all_scores.csv'
    df = pd.read_csv(path)
    df = df.reset_index(drop=True)
    

    df_black = df.filter(like='B')
    df_white = df.filter(like='W')
    df_eg = pd.concat([df.iloc[:,:4],df.filter(like='eg')],axis=1)
    df_gs = pd.concat([df.iloc[:,:4],df.filter(like='gs')],axis=1)
    
    classifier_dfs[f] = df
    dfs_b[f] = df_black
    dfs_w[f] = df_white
    dfs_eg[f] = df_eg
    dfs_gs[f] = df_gs
print(classifier_dfs['dt'])

       testB  testW  unmitB  unmitW  egdpB  egdpW  egeoB  egeoW  egtprpB  \
0      630.0    645   705.0     720  705.0    720  705.0    720    705.0   
1      565.0    764   415.0     839  415.0    839  415.0    839    415.0   
2      468.0    799   468.0     850  318.0    850  468.0    850    468.0   
3      385.0    635   385.0     710  385.0    710  385.0    635    385.0   
4      560.0    752   560.0     827  410.0    827  410.0    827    410.0   
...      ...    ...     ...     ...    ...    ...    ...    ...      ...   
12455    NaN    530     NaN     530    NaN    530    NaN    530      NaN   
12456    NaN    361     NaN     361    NaN    361    NaN    361      NaN   
12457    NaN    527     NaN     527    NaN    527    NaN    527      NaN   
12458    NaN    444     NaN     444    NaN    444    NaN    444      NaN   
12459    NaN    430     NaN     430    NaN    430    NaN    430      NaN   

       egtprpW  egfprpB  egfprpW  egerpB  egerpW  gsdpB  gsdpW  gseoB  gseoW  \
0      

### Checking if normal distributions:

if p < 0.01 (or < 0.05) then the distribution is significantly different from a normal distribution

In [19]:
for c,df in classifier_dfs.items():
    print('Classifier:',c)
    for col in df:
        data=df[col].dropna(axis=0)
        _,p = stats.kstest(data, "norm")
        if p > 0.01:
            print(col,',p:',p)
    print('Check for norm Distributions done')

Classifier: dt
Check for norm Distributions done


### Mann Whitney U test:

“a two-sample rank test for the difference between two population medians . . . It assumes that the data are independent random samples from two populations that have the same shape.”

In [20]:
mwu_path = f'{data_path}mwu/'
os.makedirs(mwu_path,exist_ok = True)

#### Variance of Distributions EG vs. GS
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [21]:
p_vals = pd.DataFrame(data={'Constraints': []})
p_signi = pd.DataFrame(data={'Constraints': []})
    
for (c1,df_eg),(c2,df_gs) in zip(dfs_eg.items(),dfs_gs.items()):
    col_signi = []
    col_vals = []
    idx = []
    
    for col_eg,col_gs in zip(df_eg,df_gs):
        
        idx.append(col_eg[2:])
        
        data_eg=df_eg[col_eg].dropna(axis=0)
        data_gs=df_gs[col_gs].dropna(axis=0)
        
        _,p = stats.mannwhitneyu(data_eg, data_gs)
        
        col_vals.append(p)
        if p< 0.05:
            col_signi.append('s')
        else:
            col_signi.append(' ')
            
    p_signi[c1] = col_signi
    p_vals[c1] = col_vals
    
p_vals['Constraints'] = idx
p_vals = p_vals.set_index('Constraints')
p_signi['Constraints'] = idx
p_signi = p_signi.set_index('Constraints')

p_vals = p_vals.round(decimals=3)
print(p_signi)
    
p_vals.to_csv(f'{mwu_path}p_eg_gs.csv')
p_signi.to_csv(f'{mwu_path}significanz_eg_gs.csv')


            dt
Constraints   
stB           
stW           
mitB          
mitW          
dpB          s
dpW          s
eoB          s
eoW           
tprpB        s
tprpW        s
fprpB        s
fprpW         
erpB          
erpW          


#### Variance of Distributions Each Model against each other
if p < 0.001 (or < 0.05) then the distributions are significantly different from each other

In [22]:
def p_values_mwu_1model(dfs,model=''):
    p_vals = pd.DataFrame(data={'Classifier': []})
    p_signi = pd.DataFrame(data={'Classifier': []})
    
    for c1,df1 in dfs.items():
        col_signi = []
        col_vals = []
        idx = []
        
        data1 = df1[model].dropna(axis=0)
        
        for c2,df2 in dfs.items():
            idx.append(c2)
            
            data2 =df2[model].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data1, data2)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c1] = col_signi
        p_vals[c1] = col_vals
        
    p_vals['Classifier'] = idx
    p_vals = p_vals.set_index('Classifier')
    p_signi['Classifier'] = idx
    p_signi = p_signi.set_index('Classifier')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_{model}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_{model}.csv')

In [23]:
for col in classifier_dfs['dt']:
    print('\nC:',col)
    p_values_mwu_1model(classifier_dfs,col)


C: testB
           dt
Classifier   
dt           

C: testW
           dt
Classifier   
dt           

C: unmitB
           dt
Classifier   
dt           

C: unmitW
           dt
Classifier   
dt           

C: egdpB
           dt
Classifier   
dt           

C: egdpW
           dt
Classifier   
dt           

C: egeoB
           dt
Classifier   
dt           

C: egeoW
           dt
Classifier   
dt           

C: egtprpB
           dt
Classifier   
dt           

C: egtprpW
           dt
Classifier   
dt           

C: egfprpB
           dt
Classifier   
dt           

C: egfprpW
           dt
Classifier   
dt           

C: egerpB
           dt
Classifier   
dt           

C: egerpW
           dt
Classifier   
dt           

C: gsdpB
           dt
Classifier   
dt           

C: gsdpW
           dt
Classifier   
dt           

C: gseoB
           dt
Classifier   
dt           

C: gseoW
           dt
Classifier   
dt           

C: gstprpB
           dt
Classifier   
dt          

#### Variance of Distributions unmitigated v Mitigated for each race

if p < 0.001 (or < 0.0005) then the distributions are significantly different from each other

In [24]:
def p_race_mwu(dfs, b_or_w = 'B'):
    p_vals = pd.DataFrame(data={'Constraints': []})
    p_signi = pd.DataFrame(data={'Constraints': []})
    
    for c,df in dfs.items():
        
        c = f'{c}{b_or_w}'
        col_signi = []
        col_vals = []
        idx = []
        
        data_unmiti = df[f'unmit{b_or_w}'].dropna(axis=0)
        
        for col in df:
            
            idx.append(col[:-1])
            
            data_miti=df[col].dropna(axis=0)
            
            _,p = stats.mannwhitneyu(data_unmiti, data_miti)
            
            col_vals.append(p)
            if p< 0.05:
                col_signi.append('s')
            else:
                col_signi.append(' ')
                
        p_signi[c] = col_signi
        p_vals[c] = col_vals
        
    p_vals['Constraints'] = idx
    p_vals = p_vals.set_index('Constraints')
    
    
    
    p_signi['Constraints'] = idx
    p_signi = p_signi.set_index('Constraints')
    
    p_vals = p_vals.round(decimals=3)
    print(p_signi)
    
    p_vals.to_csv(f'{mwu_path}p_un_vs_miti_{b_or_w}.csv')
    p_signi.to_csv(f'{mwu_path}significanz_un_vs_miti_{b_or_w}.csv')
   

In [25]:
print('Black:')
p_race_mwu(dfs_b,'B')

print('\nWhite:')
p_race_mwu(dfs_w,'W')

Black:
            dtB
Constraints    
test          s
unmit          
egdp          s
egeo          s
egtprp        s
egfprp        s
egerp          
gsdp          s
gseo           
gstprp        s
gsfprp         
gserp          

White:
            dtW
Constraints    
test          s
unmit          
egdp           
egeo           
egtprp         
egfprp         
egerp          
gsdp          s
gseo           
gstprp        s
gsfprp         
gserp          


In [ ]:
stats.pearsonr(data_unmiti, data_miti)